In [1]:
import pandas as pd
import numpy as np
import math
from math import ceil, floor
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
import os
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure

In [4]:
# remember what digits look like
digits = load_digits()
digits.data.shape

(1797, 64)

In [121]:
#convert nuclei to mnist type data

class SliceGenerator():
    def __init__(self, image_path, image_size):
        self.image_path = image_path
        self.image_size = image_size

    def slice_image(self):
        ex_nuc = pd.read_csv(self.image_path)
        ex_nuc = ex_nuc.astype(float)

        minx = np.min(ex_nuc["x"])
        miny = np.min(ex_nuc["y"])

        image_base = np.zeros((self.image_size, self.image_size))
        x_padding = (self.image_size - (np.max(ex_nuc["x"]) - np.min(ex_nuc["x"])))/2
        y_padding = (self.image_size - (np.max(ex_nuc["y"]) - np.min(ex_nuc["y"])))/2
        x_left = ceil(x_padding)
        y_left = ceil(y_padding)

        ex_nuc["new_y"] = ex_nuc["y"] - miny + y_left
        ex_nuc["new_x"] = ex_nuc["x"] - minx + x_left

        image_set = []
        for i in set(ex_nuc["z"].values): #slice
            image_base = np.zeros((self.image_size, self.image_size))
            for row in range(0, len(ex_nuc["z"])-1): #index in ex_nuc
                if ex_nuc["z"].values[row] == i: #just focussing on slice by slice
                    if int(ex_nuc["new_x"].values[row]) >= self.image_size or int(ex_nuc["new_y"].values[row]) >= self.image_size:
                        print(int(ex_nuc["new_x"].values[row])-1,int(ex_nuc["new_y"].values[row])-1)
                    image_base[int(ex_nuc["new_x"].values[row])-1,int(ex_nuc["new_y"].values[row])-1] = 1 #0 index
                else:
                    pass
            image_set.append(image_base)
        
        return image_set

    def plot_image(self):
        image_set = self.slice_image()
        fig = plt.figure(figsize=(20, 20))
        for i in range(0, len(image_set)):
            img = image_set[i]
            plt.subplot(6,6,i+1)
            plt.axis("off")
            plt.imshow(img, cmap="gray")
    
    def unravel_slices(self):
        image_set = self.slice_image()
        unravelled = []
        for array in image_set:    
            unravelled.extend(np.ravel(array))
        
        return unravelled
    
    def plot_3D(self):

        image_set = self.slice_image()
        ex = np.stack(image_set)

        # Use marching cubes to obtain the surface mesh of these ellipsoids
        verts, faces, normals, values = measure.marching_cubes_lewiner(ex, 0)

        # Display resulting triangular mesh using Matplotlib. This can also be done
        # with mayavi (see skimage.measure.marching_cubes_lewiner docstring).
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')

        # Fancy indexing: `verts[faces]` to generate a collection of triangles
        mesh = Poly3DCollection(verts[faces])
        mesh.set_edgecolor('k')
        ax.add_collection3d(mesh)


        ax.set_xlabel("x-axis: a = 6 per ellipsoid")
        ax.set_ylabel("y-axis: b = 10")
        ax.set_zlabel("z-axis: c = 16")

        ax.set_xlim(0, self.image_size)  # a = 6 (times two for 2nd ellipsoid)
        ax.set_ylim(0, self.image_size)  # b = 10
        ax.set_zlim(0, self.image_size)  # c = 16


        plt.tight_layout()
        base = os.path.basename(os.path.normpath(self.image_path))
        plt.savefig(base+".png")
        plt.close(fig)

In [122]:
i = SliceGenerator("/Users/esthomas/Mirror/Andor_Rotation/github_repo/3D_cells/G05_multiOrganelles_Linked/0_prediction_c0.model.p_cell_1_coordinates.csv", 50)
i.plot_3D()

/var/folders/p1/99sndrn56lv71k57w35yh8n40000gn/T/ipykernel_9520/2612188475.py:61: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  verts, faces, normals, values = measure.marching_cubes_lewiner(ex, 0)


In [65]:
# feed my data into umap and see what happens
# I don't agree that this is the right way to do it - must lose some structure but let's see

parent = "/Users/esthomas/Mirror/Andor_Rotation/github_repo/3D_cells/G05_multiOrganelles_Linked"
data = []
for filename in os.listdir(parent):
    if "0_prediction_c0" in filename:
        img = SliceGenerator(parent+"/"+filename, 100)
        data.append(img.unravel_slices())


In [72]:
import csv

with open("nuclei.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(data)

In [73]:
import umap

In [85]:
# make all arrays the same size - by appending zeros
import itertools
from pprint import pprint

pad_token = 0.0

padded = zip(*itertools.zip_longest(*data, fillvalue=pad_token))
padded_data = list(padded)


In [86]:
import csv

with open("padded_nuclei.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(padded_data)

In [87]:
p_data = np.asarray(padded_data)

In [88]:
p_data.shape

(819, 600000)

In [89]:
embedding = umap.UMAP(n_neighbors=5).fit_transform(p_data)

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*embedding.T, s=1, cmap='Spectral', alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
#cbar = plt.colorbar(boundaries=np.arange(11)-0.5)
#cbar.set_ticks(np.arange(10))
#cbar.set_ticklabels(classes)
plt.title('Fashion MNIST Embedded via UMAP');

In [92]:
embedding2 = umap.UMAP(n_neighbors=5).fit_transform(p_data)

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(*embedding2.T, s=1, cmap='Spectral', alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
#cbar = plt.colorbar(boundaries=np.arange(11)-0.5)
#cbar.set_ticks(np.arange(10))
#cbar.set_ticklabels(classes)
plt.title('Fashion MNIST Embedded via UMAP');

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import Spectral10

output_notebook()

In [96]:
from io import BytesIO
from PIL import Image
import base64

In [ ]:
def embeddable_image(data):
    img_data = 255 - 15 * data.astype(np.uint8)
    image = Image.fromarray(img_data, mode='L').resize((64, 64), Image.BICUBIC)
    buffer = BytesIO()
    image.save(buffer, format='png')
    for_encoding = buffer.getvalue()
    return 'data:image/png;base64,' + base64.b64encode(for_encoding).decode()

In [116]:
def embeddable_image(image_path):
    base = os.path.basename(os.path.normpath(image_path))
    if base+".png" in os.list: 
    i = SliceGenerator(image_path, 100)
    i.plot_3D()
    image = Image.open(base+".png")
    buffer = BytesIO()
    image.save(buffer, format='png')
    for_encoding = buffer.getvalue()
    return 'data:image/png;base64,' + base64.b64encode(for_encoding).decode()

In [ ]:
embeddable_image(parent+"/"+filename)

In [119]:
image_path_list = []
parent = "/Users/esthomas/Mirror/Andor_Rotation/github_repo/3D_cells/G05_multiOrganelles_Linked"

for filename in os.listdir(parent):
    if "0_prediction_c0" in filename:
        image_path_list.append(parent+"/"+filename)



In [ ]:
digits_df = pd.DataFrame(embedding, columns=('x', 'y'))
# digits_df['digit'] = [str(x) for x in digits.target]
digits_df['image'] = list(map(embeddable_image, image_path_list))

datasource = ColumnDataSource(digits_df)
# color_mapping = CategoricalColorMapper(factors=[str(9 - x) for x in digits.target_names],
                                    #    palette=Spectral10)

plot_figure = figure(
    title='UMAP projection of the Digits dataset',
    plot_width=600,
    plot_height=600,
    tools=('pan, wheel_zoom, reset')
)

plot_figure.add_tools(HoverTool(tooltips="""
<div>
    <div>
        <img src='@image' style='float: left; margin: 5px 5px 5px 5px'/>
    </div>
    <div>
        <span style='font-size: 16px; color: #224499'>Digit:</span>
        <span style='font-size: 18px'>@digit</span>
    </div>
</div>
"""))

plot_figure.circle(
    'x',
    'y',
    source=datasource,
    # color=dict(field='digit', transform=color_mapping),
    line_alpha=0.6,
    fill_alpha=0.6,
    size=4
)
show(plot_figure)

In [ ]:
digits_df = pd.DataFrame(embedding, columns=('x', 'y'))
# digits_df['digit'] = [str(x) for x in digits.target]
digits_df['image'] = list(map(embeddable_image, image_path_list))

datasource = ColumnDataSource(digits_df)
# color_mapping = CategoricalColorMapper(factors=[str(9 - x) for x in digits.target_names],
                                    #    palette=Spectral10)

plot_figure = figure(
    title='UMAP projection of the Digits dataset',
    plot_width=600,
    plot_height=600,
    tools=('pan, wheel_zoom, reset')
)

plot_figure.add_tools(HoverTool(tooltips="""
<div>
    <div>
        <img src='@image' style='float: left; margin: 5px 5px 5px 5px'/>
    </div>
    <div>
        <span style='font-size: 16px; color: #224499'>Digit:</span>
        <span style='font-size: 18px'>@digit</span>
    </div>
</div>
"""))

plot_figure.circle(
    'x',
    'y',
    source=datasource,
    # color=dict(field='digit', transform=color_mapping),
    line_alpha=0.6,
    fill_alpha=0.6,
    size=4
)
show(plot_figure)